# Implied Returns & Black-Litterman

### Black-Litterman model 
The BL model uses "views" from the active investor and incorporates them with the implied returns calculated from the value-weighted market portfolio. The model uses the implied returns as a prior distribution and treats the "views" as observed data used to calculate a posterior distribution on the returns as well as a posterior for the covariance of the returns.

### $$\pi = \delta\Sigma w$$
$\pi$ : implied returns (N x 1)

$\delta$ : risk aversion parameter (1 x 1)
- Original paper uses a value of 2.5 for this parameter
- Another approach is the set it to the market price of risk:
    - $\delta = \mu_M / \sigma^2_M$

$\Sigma$ : Covariance of the assets (N x N)

$w$ : Equilibrium market weights (N x 1)
        
### Additional variables

$Q$ : (K x 1) Matrix containing views

$P$ : (K x N) Pick matrix. Links each view to the corresponding asset

$\Omega$ : Covariance matrix representing the uncertaintly of views

$\tau$ : A scalar indicating the uncertainty of the prior
- Original paper suggests to use a small number, like $\tau = 1/T$
    - $T$ = number of periods of data used
    - Example: 5 years of data - > $T = 1/(5 \cdot 12) = 0.02$

### How to represent views in $Q$ and $P$ : 
For your $k$-th view, set $Q_k$ to be the difference of returns between assets i and j. Set $P_ki$ to -1 for the asset with the lower expected return (asset i). Set $P_kj$ to 1 for the asset with the higher expected return (asset j). All other elements in row $k$ should be set to 0. $\Omega$ represents the uncertaintly of the views. The original paper recommends setting $\Omega = diag(P (\tau \Sigma) P^T)$.

### Equations for retrieving the posterior distribution on the returns as well as the covariance of the returns.

\begin{equation}
\label{eq:blMu}
\mu^{BL} = \pi + \tau \Sigma P^T[(P \tau \Sigma P^T) + \Omega]^{-1}[Q - P \pi]
\end{equation}

\begin{equation}
\label{eq:blSigma}
\Sigma^{BL} = \Sigma + \tau \Sigma - \tau\Sigma P^T(P \tau \Sigma P^T + \Omega)^{-1} P \tau \Sigma
\end{equation}

# Use

### For the examples in this notebook, use data from 1980 to 2015 for the following 5 industries:
- Beer, Hlth, Fin, Softw, and Clths

### Calculate the implied annual returns using the with a delta of 2.5 and the same sigma prior method as the He-Litterman paper
- Neutral prior distribution on expected returns is obtained by reverse engineering, assuming that the cap-weighted market portfolio is the optimal portfolio.
- Covariance matrix & weights -> implied returns


In [1]:
import numpy as np
import pandas as pd
import edhec_risk_kit as erk

In [2]:
industries = ['Beer', 'Hlth', 'Fin', 'Softw', 'Clths']
ind_rets = erk.get_ind_returns(n_inds=49).loc['1980':'2015', industries]
ind_mcap = erk.get_ind_market_caps(49).loc['1980':'2015', industries]
weights = ind_mcap.loc['1980-01'] / ind_mcap.loc['1980-01'].sum()
delta = 2.5

### Use equation $\pi = \delta\Sigma w$
erk.implied_returns(delta, sigma, w)

In [3]:
s = pd.DataFrame(ind_rets.cov()*np.sqrt(12), index=industries, columns=industries)
pi = erk.implied_returns(delta=2.5, sigma=s, w=weights)
pi

Beer     0.013814
Hlth     0.020526
Fin      0.032345
Softw    0.029059
Clths    0.023959
Name: Implied Returns, dtype: float64

### Implement the active view that Softw will outperform Fin and Clths by 1%
- What will be the entry in the P matrix for Fin? For Softw?

### Use equation $\Omega = diag(P (\tau \Sigma) P^T)$
erk.proportional_prior(sigma, tau, p)
- input into the BL equations

### Use the Black-Litterman equations

\begin{equation}
\label{eq:blMu}
\mu^{BL} = \pi + \tau \Sigma P^T[(P \tau \Sigma P^T) + \Omega]^{-1}[Q - P \pi]
\end{equation}

\begin{equation}
\label{eq:blSigma}
\Sigma^{BL} = \Sigma + \tau \Sigma - \tau\Sigma P^T(P \tau \Sigma P^T + \Omega)^{-1} P \tau \Sigma
\end{equation}

erk.bl(w_prior, sigma_prior, p, q, omega, delta, tau)

In [4]:
q = pd.Series([.01]) # one view
p = pd.DataFrame([0.]*len(industries), index=industries).T
w_Fin =  weights.loc["Fin"]/(weights.loc["Fin"]+weights.loc["Clths"])
w_Clths =  weights.loc["Clths"]/(weights.loc["Fin"]+weights.loc["Clths"])
p.iloc[0]['Softw'] = 1.
p.iloc[0]['Fin'] = -w_Fin
p.iloc[0]['Clths'] = -w_Clths
print('P:\n', p)

P:
    Beer  Hlth       Fin  Softw     Clths
0   0.0   0.0 -0.890879    1.0 -0.109121


### Implement the active view that Softw will outperform Fin and Clths by 1%
- What will the new expected returns be?

In [5]:
delta = 2.5
tau = 0.05
sigma_prior = ind_rets.cov()*np.sqrt(12)
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = erk.bl(weights, sigma_prior, p, q, tau = tau)
bl_mu

Beer     0.013861
Hlth     0.020609
Fin      0.031688
Softw    0.034657
Clths    0.023933
dtype: float64

### Calculate the weights in the Max Sharpe Ratio portfolio given these new expected returns

In [6]:
pd.DataFrame({
    'Weight':erk.msr(riskfree_rate=0.005, er=bl_mu, cov=bl_sigma)
    },
    index=industries
)

,Weight
Beer,0.000000
Hlth,0.002034
Fin,0.712257
Softw,0.253155
Clths,0.032554


### Change the view to be that you believe Software will outperform Finance and Clothes by 4%
- Then calculate the new weights of the Max Sharpe Ratio portfolio

In [7]:
q = pd.Series([.04]) # one view
p = pd.DataFrame([0.]*len(industries), index=industries).T
w_Fin =  weights.loc["Fin"]/(weights.loc["Fin"]+weights.loc["Clths"])
w_Clths =  weights.loc["Clths"]/(weights.loc["Fin"]+weights.loc["Clths"])
p.iloc[0]['Softw'] = 1.
p.iloc[0]['Fin'] = -w_Fin
p.iloc[0]['Clths'] = -w_Clths
# P matrix doesn't change
print('P:\n', p)

P:
    Beer  Hlth       Fin  Softw     Clths
0   0.0   0.0 -0.890879    1.0 -0.109121


In [8]:
delta = 2.5
tau = 0.05
sigma_prior = ind_rets.cov()*np.sqrt(12)
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = erk.bl(weights, sigma_prior, p, q, tau = tau)
print('New expected returns:')
bl_mu

New expected returns:


Beer     0.013973
Hlth     0.020810
Fin      0.030095
Softw    0.048231
Clths    0.023871
dtype: float64

In [9]:
print('New MSR weights')
pd.DataFrame({
    'Weight':erk.msr(riskfree_rate=0.005, er=bl_mu, cov=bl_sigma)
    },
    index=industries
)

New MSR weights


,Weight
Beer,0.000000e+00
Hlth,3.603698e-18
Fin,1.573941e-01
Softw,8.426059e-01
Clths,0.000000e+00
